In [ ]:
entities = []
# for i in range(5):
for page in dataset["train"][-1]["Documents"][0]["Pages"]:
  # Check if there's any actual content in the page, if not, don't call the API
  if page["Page Text"] == "":
    continue
  # When we're clear on the schema, replace everything after the line "fill out the schema in system role"
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": [
          {
            "text": "Your role is to take look at the following schema and text chunk, and then fill out the schema:\n\nSchema\n{\n  \"$schema\": \"http://json-schema.org/draft-07/schema#\",\n  \"type\": \"object\",\n  \"properties\": {\n    \"Project\": {\n      \"type\": \"object\",\n      \"properties\": {\n        \"id\": { \"type\": \"string\" },\n        \"name\": { \"type\": \"string\" },\n        \"location\": { \"type\": \"string\" },\n        \"date\": { \"type\": \"string\" },\n        \"summary\": { \"type\": \"string\" },\n        \"capacity\": { \"type\": \"string\" },\n        \"technology\": { \"type\": \"string\" }\n      },\n      \"required\": [\"id\", \"name\", \"location\", \"summary\", \"capacity\", \"technology\"]\n    },\n    \"Person\": {\n      \"type\": \"object\",\n      \"properties\": {\n        \"name\": { \"type\": \"string\" },\n        \"email\": { \"type\": \"string\", \"format\": \"email\" },\n        \"phone\": { \"type\": \"string\", \"format\": \"phone\" },\n        \"role\": { \"type\": \"string\" }\n      },\n      \"required\": [\"name\", \"email\", \"phone\", \"role\"]\n    },\n    \"Organizations\": {\n      \"type\": \"array\",\n      \"items\": {\n        \"type\": \"object\",\n        \"properties\": {\n          \"name\": { \"type\": \"string\" },\n          \"type\": { \"type\": \"string\" },\n          \"acronym\": { \"type\": \"string\" },\n          \"fullName\": { \"type\": \"string\" },\n          \"role\": { \"type\": \"string\" },\n          \"isApplicant\": { \"type\": \"boolean\" }\n        },\n        \"required\": [\"name\", \"type\", \"role\", \"isApplicant\", \"acronym\", \"fullName\"]\n      }\n    },\n    \"Comment\": {\n      \"type\": \"object\",\n      \"properties\": {\n        \"id\": { \"type\": \"string\" },\n        \"content\": { \"type\": \"string\" },\n        \"date\": { \"type\": \"string\" },\n        \"sentiment\": { \"type\": \"string\" }\n      }\n    }\n  },\n  \"required\": [\"Project\", \"Person\", \"Organizations\", \"Comment\"]\n}\n\nOther instructions:\n\n- Extract as much information as possible from the text chunk to fill the schema.\n- If there's not enough information for a field, use an empty string or null.\n- For the Project, include details about capacity and technology if available.\n- For Person, try to infer their role based on the context.\n- For Organizations:\n  - Create an entry for each organization mentioned, including the applicant and any regulatory bodies.\n  - Set 'isApplicant' to true for the company proposing the project.\n  - Pay special attention to acronyms:\n    - If an organization is mentioned with its full name and acronym (e.g., \"United States Army Corps of Engineers (USACE)\"), capture both in the respective fields.\n    - If only an acronym is given, attempt to infer the full name if possible, or leave the 'fullName' field empty if uncertain.\n    - If only the full name is given, attempt to create an acronym using the initial letters of major words in the name.\n  - For 'type', categorize as \"Federal\", \"State\", \"Local\", \"Private\", or \"Non-profit\" based on context.\n  - For 'role', use descriptors like \"Applicant\", \"Regulatory Body\", \"Contractor\", etc.\n- If no comment is provided in the text, leave the Comment object empty.\n- Pay attention to details about permits, environmental impacts, and involved parties.\n\nReturn your results as a JSON object, ensuring all required fields are present.",
            "type": "text"
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "text": page["Page Text"],
            "type": "text"
          }
        ]
      }
    ],
    temperature=1,
    max_tokens=1559,
    top_p=1,
    frequency_penalty=0,
    response_format={ "type": "json_object"},
    presence_penalty=0
    )
  output = json.loads(response.choices[0].message.content)
  entities.append(output)
  print("Successfully called the OpenAI API for Project: ", end="")
  print(entities[-1].get("Project", {}).get("name", "No Project name found"))
